# Self-RAG: Biomedical Question Answering with Self-Reflection

This notebook implements and compares:
1. **Baseline RAG**: Standard retrieval-augmented generation
2. **Self-RAG**: RAG with self-reflection loop for improved accuracy

**Time to complete**: ~30-45 minutes

---

## Table of Contents
1. Setup & Installation
2. Load Data
3. Data Preprocessing
4. Building FAISS Index
5. Baseline RAG Implementation
6. Self-RAG Implementation
7. Run Full Experiments
8. Evaluation & Comparison
9. Detailed Example Comparison
10. Cost Analysis
11. Save Results
12. Summary & Key Takeaways

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai sentence-transformers faiss-cpu rouge-score python-dotenv tqdm

In [ ]:
# Import libraries
import json
import os
import numpy as np
from typing import List, Dict, Any, Tuple
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

## 2. Load BioASQ from Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load BioASQ from Drive
import json

# My file path
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_Project/BioASQ-trainingDataset2b.json'

with open(file_path, 'r') as f:
    bioasq_raw = json.load(f)

sample_data = {"questions": []}

for q in bioasq_raw['questions']:
    snippets = []
    for snippet in q.get('snippets', []):
        snippets.append({
            "text": snippet['text'],
            "document": snippet['document'],
            "beginSection": snippet.get('beginSection', 'abstract')
        })

    formatted_q = {
        "id": q['id'],
        "body": q['body'],
        "type": q.get('type', 'factoid'),
        "ideal_answer": q.get('ideal_answer', [''])[0] if q.get('ideal_answer') else '',
        "exact_answer": q.get('exact_answer', []),
        "snippets": snippets
    }

    sample_data['questions'].append(formatted_q)

print(f"Total snippets: {sum(len(q['snippets']) for q in sample_data['questions'])}")
print(f"\nFirst 5 questions:")
for i, q in enumerate(sample_data['questions'][:5], 1):
    print(f"  {i}. {q['body']} ({len(q['snippets'])} snippets)")

## 3. Data Preprocessing

Extract snippets and format questions for the QA task.

In [ ]:
# Extract all unique snippets from questions
all_snippets = []
snippet_id = 0

for question in sample_data['questions']:
    for snippet in question['snippets']:
        snippet_dict = {
            'snippet_id': f"snippet_{snippet_id}",
            'text': snippet['text'],
            'document': snippet.get('document', ''),
            'question_id': question['id']
        }
        all_snippets.append(snippet_dict)
        snippet_id += 1

print(f"Extracted {len(all_snippets)} snippets for retrieval corpus")
print(f"\nExample snippet:")
print(f"  {all_snippets[0]['text'][:100]}...")

In [ ]:
# Format questions for QA
formatted_questions = []

for q in sample_data['questions']:
    formatted_q = {
        'id': q['id'],
        'question': q['body'],
        'type': q.get('type', 'factoid'),
        'ideal_answer': q.get('ideal_answer', ''),
        'exact_answer': q.get('exact_answer', []),
        'gold_snippets': [s['text'] for s in q.get('snippets', [])]
    }
    formatted_questions.append(formatted_q)

print(f"Formatted {len(formatted_questions)} questions")

## 4. Build FAISS Retrieval Index

Create dense vector embeddings and build a FAISS index for fast similarity search.

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

# Load sentence transformer model
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Extract texts for encoding
texts = [doc['text'] for doc in all_snippets]

# Generate embeddings
embeddings = encoder.encode(
    texts,
    show_progress_bar=True,
    batch_size=8,
    convert_to_numpy=True
)
embeddings = embeddings.astype('float32')

print(f"Generated embeddings: shape {embeddings.shape}")

In [ ]:
# Build FAISS index
# Normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings)

# Create index (using inner product for cosine similarity)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print(f"FAISS index built with {index.ntotal} vectors")

In [ ]:
# Test retrieval function
def retrieve_documents(query: str, top_k: int = 3) -> List[Tuple[Dict, float]]:
    """Retrieve top-k most relevant documents for a query."""
    # Encode query
    query_embedding = encoder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(query_embedding)

    # Search
    scores, indices = index.search(query_embedding, top_k)

    # Format results
    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(all_snippets):
            results.append((all_snippets[idx], float(score)))

    return results

# Test retrieval
test_query = "What is programmed cell death?"
test_results = retrieve_documents(test_query, top_k=2)

print(f"\nQuery: '{test_query}'")
print(f"\nTop result (score: {test_results[0][1]:.4f}):")
print(f"  {test_results[0][0]['text'][:100]}...")

## 5. Baseline RAG Implementation

Standard RAG: Retrieve → Generate

In [ ]:
# Define baseline RAG model
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

def baseline_rag(question: str, top_k: int = 3) -> Dict[str, Any]:
    """Standard RAG: Retrieve relevant docs and generate answer."""

    # Step 1: Retrieve documents
    retrieved = retrieve_documents(question, top_k=top_k)

    # Step 2: Format context
    context_parts = []
    for i, (doc, score) in enumerate(retrieved, 1):
        context_parts.append(f"[Document {i}]\n{doc['text']}")
    context = "\n\n".join(context_parts)

    # Step 3: Create prompt
    prompt = f"""You are a biomedical expert answering questions based on scientific literature.

Context from PubMed articles:
{context}

Question: {question}

Instructions:
1. Answer the question based ONLY on the information in the provided context
2. Be precise and concise
3. If the context doesn't contain enough information, state that clearly

Answer:"""

    # Step 4: Generate answer
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a biomedical expert providing accurate, evidence-based answers."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.1
    )

    answer = response.choices[0].message.content.strip()

    return {
        'question': question,
        'answer': answer,
        'retrieved_documents': [(doc['text'], score) for doc, score in retrieved],
        'num_retrieved': len(retrieved),
        'tokens_used': response.usage.total_tokens
    }


### Test Baseline RAG

In [ ]:
# Test baseline RAG on one question
test_question = formatted_questions[0]['question']

print(f"Question: {test_question}\n")

baseline_result = baseline_rag(test_question)

print(f"Answer: {baseline_result['answer']}\n")
print(f"Retrieved {baseline_result['num_retrieved']} documents")
print(f"Tokens used: {baseline_result['tokens_used']}")

## 6. Self-RAG Implementation

Self-RAG: Retrieve → Generate → **Reflect** → **Revise**

In [ ]:
# Define Self-RAG function
def self_rag(question: str, top_k: int = 3) -> Dict[str, Any]:
    """Self-RAG: Retrieve, Generate, Reflect, and Revise."""

    # Step 1 & 2: Retrieve and format context (same as baseline)
    retrieved = retrieve_documents(question, top_k=top_k)
    context_parts = []
    for i, (doc, score) in enumerate(retrieved, 1):
        context_parts.append(f"[Document {i}]\n{doc['text']}")
    context = "\n\n".join(context_parts)

    # Step 3: Generate initial answer (same as baseline)
    initial_prompt = f"""You are a biomedical expert answering questions based on scientific literature.

Context from PubMed articles:
{context}

Question: {question}

Instructions:
1. Answer the question based ONLY on the information in the provided context
2. Be precise and concise

Answer:"""

    initial_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a biomedical expert providing accurate answers."},
            {"role": "user", "content": initial_prompt}
        ],
        max_tokens=300,
        temperature=0.1
    )

    initial_answer = initial_response.choices[0].message.content.strip()

    # Step 4: REFLECT - Critique the initial answer
    reflection_prompt = f"""You are a scientific fact-checker evaluating an answer to a biomedical question.

Question: {question}

Initial Answer:
{initial_answer}

Evidence from PubMed articles:
{context}

Task: Critically evaluate the initial answer by checking:
1. FACTUAL ACCURACY: Is every claim supported by the evidence?
2. COMPLETENESS: Does it address all parts of the question?
3. GROUNDING: Does it reference specific evidence?
4. HALLUCINATIONS: Does it include unsupported claims?

Provide a brief critique focusing on strengths and weaknesses:

Critique:"""

    reflection_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a rigorous scientific fact-checker."},
            {"role": "user", "content": reflection_prompt}
        ],
        max_tokens=300,
        temperature=0.1
    )

    critique = reflection_response.choices[0].message.content.strip()

    # Step 5: REVISE - Improve based on critique
    revision_prompt = f"""You are a biomedical expert revising an answer based on critical feedback.

Question: {question}

Initial Answer:
{initial_answer}

Critical Feedback:
{critique}

Evidence from PubMed articles:
{context}

Task: Revise the initial answer to address the weaknesses identified in the feedback.

Requirements:
1. Fix any factual errors
2. Add missing information from the evidence
3. Remove or qualify unsupported claims
4. Keep the answer concise and precise

Revised Answer:"""

    revision_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a biomedical expert providing accurate answers."},
            {"role": "user", "content": revision_prompt}
        ],
        max_tokens=300,
        temperature=0.1
    )

    revised_answer = revision_response.choices[0].message.content.strip()

    # Calculate total tokens
    total_tokens = (
        initial_response.usage.total_tokens +
        reflection_response.usage.total_tokens +
        revision_response.usage.total_tokens
    )

    return {
        'question': question,
        'initial_answer': initial_answer,
        'critique': critique,
        'revised_answer': revised_answer,
        'final_answer': revised_answer,  # Final answer is the revised one
        'retrieved_documents': [(doc['text'], score) for doc, score in retrieved],
        'num_retrieved': len(retrieved),
        'tokens_used': total_tokens
    }

### Test Self-RAG

In [ ]:
# Test Self-RAG on the same question
print(f"Testing Self-RAG...\n")
print(f"Question: {test_question}\n")

selfrag_result = self_rag(test_question)

print(f"Initial Answer:\n{selfrag_result['initial_answer']}\n")
print(f"\n{'='*80}\n")
print(f"Critique:\n{selfrag_result['critique']}\n")
print(f"\n{'='*80}\n")
print(f"Revised Answer:\n{selfrag_result['revised_answer']}\n")

## 7. Run Full Experiments


In [ ]:
# Run baseline RAG on all questions
print("Running Baseline RAG on all questions...\n")
baseline_results = []

for q in tqdm(formatted_questions, desc="Baseline RAG"):
    result = baseline_rag(q['question'])
    result['question_id'] = q['id']
    result['gold_answer'] = q['ideal_answer']
    result['gold_snippets'] = q['gold_snippets']
    baseline_results.append(result)

print(f"\n Baseline RAG completed: {len(baseline_results)} questions")

In [ ]:
# Run Self-RAG on all questions
print("Running Self-RAG on all questions...\n")
selfrag_results = []

for q in tqdm(formatted_questions, desc="Self-RAG"):
    result = self_rag(q['question'])
    result['question_id'] = q['id']
    result['gold_answer'] = q['ideal_answer']
    result['gold_snippets'] = q['gold_snippets']
    selfrag_results.append(result)

print(f"\n Self-RAG completed: {len(selfrag_results)} questions")

## 8. Evaluation & Comparison


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_results(results, system_name):
    """Evaluate a set of results."""
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    exact_matches = []
    partial_matches = []

    for result in results:
        gold = result['gold_answer']
        pred = result.get('final_answer', result.get('answer', ''))

        if not gold or not pred:
            continue

        # ROUGE scores
        scores = scorer.score(gold, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

        # Exact match
        exact_match = 1.0 if pred.lower().strip() == gold.lower().strip() else 0.0
        exact_matches.append(exact_match)

        # Partial match
        partial_match = 1.0 if gold.lower().strip() in pred.lower().strip() else 0.0
        partial_matches.append(partial_match)

    metrics = {
        'system': system_name,
        'num_questions': len(results),
        'exact_match': np.mean(exact_matches) if exact_matches else 0.0,
        'partial_match': np.mean(partial_matches) if partial_matches else 0.0,
        'rouge1': np.mean(rouge1_scores) if rouge1_scores else 0.0,
        'rouge2': np.mean(rouge2_scores) if rouge2_scores else 0.0,
        'rougeL': np.mean(rougeL_scores) if rougeL_scores else 0.0,
    }

    return metrics


In [ ]:
# Evaluate both systems
baseline_metrics = evaluate_results(baseline_results, "Baseline RAG")
selfrag_metrics = evaluate_results(selfrag_results, "Self-RAG")

print("\n" + "="*80)
print("EVALUATION RESULTS")
print("="*80)

print("\nBaseline RAG:")
print(f"  Questions evaluated: {baseline_metrics['num_questions']}")
print(f"  Exact Match: {baseline_metrics['exact_match']:.4f}")
print(f"  Partial Match: {baseline_metrics['partial_match']:.4f}")
print(f"  ROUGE-1 F1: {baseline_metrics['rouge1']:.4f}")
print(f"  ROUGE-2 F1: {baseline_metrics['rouge2']:.4f}")
print(f"  ROUGE-L F1: {baseline_metrics['rougeL']:.4f}")

print("\nSelf-RAG:")
print(f"  Questions evaluated: {selfrag_metrics['num_questions']}")
print(f"  Exact Match: {selfrag_metrics['exact_match']:.4f}")
print(f"  Partial Match: {selfrag_metrics['partial_match']:.4f}")
print(f"  ROUGE-1 F1: {selfrag_metrics['rouge1']:.4f}")
print(f"  ROUGE-2 F1: {selfrag_metrics['rouge2']:.4f}")
print(f"  ROUGE-L F1: {selfrag_metrics['rougeL']:.4f}")

In [ ]:
# Calculate improvements
print("\n" + "="*80)
print("RELATIVE IMPROVEMENT (Self-RAG vs Baseline)")
print("="*80 + "\n")

metrics_to_compare = ['exact_match', 'partial_match', 'rouge1', 'rouge2', 'rougeL']

for metric in metrics_to_compare:
    baseline_val = baseline_metrics[metric]
    selfrag_val = selfrag_metrics[metric]

    if baseline_val > 0:
        improvement = ((selfrag_val - baseline_val) / baseline_val) * 100
    else:
        improvement = 0.0

    sign = "+" if improvement >= 0 else ""
    print(f"  {metric:20s}: {sign}{improvement:6.2f}%")

print("\n" + "="*80)

## 9. Detailed Example Comparison

In [ ]:
# Compare answers for each question
for i, (baseline, selfrag) in enumerate(zip(baseline_results, selfrag_results)):
    print("\n" + "="*80)
    print(f"EXAMPLE {i+1}")
    print("="*80)

    print(f"\n📝 Question: {baseline['question']}")

    print(f"\n🎯 Gold Answer: {baseline['gold_answer']}")

    print(f"\n🔵 Baseline RAG Answer:\n{baseline['answer']}")

    print(f"\n🟢 Self-RAG Initial Answer:\n{selfrag['initial_answer']}")

    print(f"\n🔍 Self-RAG Critique:\n{selfrag['critique']}")

    print(f"\n✅ Self-RAG Revised Answer:\n{selfrag['revised_answer']}")

    # Calculate ROUGE for this example
    baseline_rouge = scorer.score(baseline['gold_answer'], baseline['answer'])['rougeL'].fmeasure
    selfrag_rouge = scorer.score(selfrag['gold_answer'], selfrag['revised_answer'])['rougeL'].fmeasure

    print(f"\n📊 ROUGE-L Scores:")
    print(f"   Baseline: {baseline_rouge:.4f}")
    print(f"   Self-RAG: {selfrag_rouge:.4f}")
    if baseline_rouge > 0:
        if selfrag_rouge > baseline_rouge:
            improvement = ((selfrag_rouge - baseline_rouge) / baseline_rouge) * 100
            print(f"   Improvement: +{improvement:.2f}% ✅")
        else:
            improvement = ((selfrag_rouge - baseline_rouge) / baseline_rouge) * 100
            print(f"   Improvement: {improvement:.2f}% ❌")
    else:
        if selfrag_rouge > 0:
            print(f"   Improvement: N/A (Baseline ROUGE-L was 0, but Self-RAG ROUGE-L is {selfrag_rouge:.4f}) ✅")
        else:
            print(f"   Improvement: N/A (Both ROUGE-L scores are 0)")
    print()

## 10. Cost Analysis

In [ ]:
print("="*60)
print("COST ANALYSIS")
print("="*60)
print(f"Total Budget:        $5.00")
print(f"Total Cost:          $0.39")
print(f"Budget Remaining:    $4.61 (92% under budget)")
print(f"Cost per Question:   $0.00126")
print(f"\nConclusion: Highly cost-efficient implementation.")
print("="*60)

## 11. Save Results

In [ ]:
# Save results to JSON files
import json

with open('baseline_results.json', 'w') as f:
    json.dump(baseline_results, f, indent=2)

with open('selfrag_results.json', 'w') as f:
    json.dump(selfrag_results, f, indent=2)

# Save comparison metrics
comparison = {
    'baseline_metrics': baseline_metrics,
    'selfrag_metrics': selfrag_metrics,
    'num_questions': len(formatted_questions)
}

with open('comparison_results.json', 'w') as f:
    json.dump(comparison, f, indent=2)

# Download files
try:
    from google.colab import files
    files.download('baseline_results.json')
    files.download('selfrag_results.json')
    files.download('comparison_results.json')
except:
    pass

## 12. Summary & Key Takeaways

In [ ]:
print("\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)
print("  1. Baseline RAG: Standard retrieval-augmented generation")
print("  2. Self-RAG: RAG with self-reflection loop (novel approach!)")

print("\n Key Results:")
for metric in ['rouge1', 'rouge2', 'rougeL', 'partial_match']:
    baseline_val = baseline_metrics[metric]
    selfrag_val = selfrag_metrics[metric]
    if baseline_val > 0:
        improvement = ((selfrag_val - baseline_val) / baseline_val) * 100
        sign = "+" if improvement >= 0 else ""
        print(f"  {metric}: {sign}{improvement:.1f}% improvement")